In [1]:
# notebook settings
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import sympy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from copy import copy
from math import floor

# internal imports
from pympc.geometry.polyhedron import Polyhedron
from pympc.dynamics.discrete_time_systems import LinearSystem, AffineSystem, PieceWiseAffineSystem
from pympc.control.hybrid_benchmark.controllers import HybridModelPredictiveController
from pympc.plot import plot_input_sequence, plot_state_trajectory, plot_output_trajectory
from pympc.control.hybrid_benchmark.utils import get_constraint_set, remove_redundant_inequalities_fast, convex_hull_method_fast

# Problem set-up

In [2]:
# numeric parameters of the system
m = 1.
r = .1
I = .4*m*r**2.
d = .4
l = .3
mu = .2
g = 10.
h = .05

In [3]:
# symbolic state
xb, yb, tb = sp.symbols('xb yb tb') # position of the ball
xf, yf = sp.symbols('xf yf') # position of the floor
xdb, ydb, tdb = sp.symbols('xdb ydb tdb') # velocity of the ball
xdf, ydf = sp.symbols('xdf ydf') # velocity of the floor
x = sp.Matrix([
    xb, yb, tb,
    xf, yf,
    xdb, ydb, tdb,
    xdf, ydf
])

# symbolic input
xd2f, yd2f = sp.symbols('xd2f yd2f') # acceleration of the floor
u = sp.Matrix([
    xd2f, yd2f
])

# contact forces
ftf, fnf = sp.symbols('ftf fnf') # floor force
ftc, fnc = sp.symbols('ftc fnc') # ceiling force

In [4]:
# ball velocity update
xdb_next = xdb + h*ftf/m - h*ftc/m
ydb_next = ydb + h*fnf/m - h*fnc/m - h*g
tdb_next = tdb + r*h*ftf/I + r*h*ftc/I

# ball position update
xb_next = xb + h*xdb_next
yb_next = yb + h*ydb_next
tb_next = tb + h*tdb_next

# floor velocity update
xdf_next = xdf + h*xd2f
ydf_next = ydf + h*yd2f

# floor position update
xf_next = xf + h*xdf_next
yf_next = yf + h*ydf_next

# state update
x_next = sp.Matrix([
    xb_next, yb_next, tb_next,
    xf_next, yf_next,
    xdb_next, ydb_next, tdb_next,
    xdf_next, ydf_next
])

In [5]:
# relative tangential velocity
sliding_velocity_floor = xdb_next + r*tdb_next - xdf_next
sliding_velocity_ceiling = xdb_next - r*tdb_next

# gap function floor
gap_floor = yb_next - yf_next

# gap function ceiling
gap_ceiling = d - 2.*r - yb_next

# ball distance to boundaries
ball_on_floor = sp.Matrix([
    xb_next - xf_next - l,
    xf_next - xb_next - l
])
ball_on_ceiling = sp.Matrix([
    xb_next - l,
    - xb_next - l
])

In [6]:
# state bounds
x_max = np.array([
    l, d-2.*r, 1.2*np.pi, # ball config
    l, d-2.*r-.05,            # floor config
    2., 2., 10.,     # ball vel
    2., 2.          # floor vel
])
x_min = - x_max

# input bounds
u_max = np.array([
    30., 30.,          # floor acc
])
u_min = - u_max

# domain bounds
xu = x.col_join(u)
xu_min = np.concatenate((x_min, u_min))
xu_max = np.concatenate((x_max, u_max))

In [7]:
# discrete time dynamics in mode 1
# (ball in the air)

# set forces to zero
f_m1 = {ftf: 0., fnf: 0., ftc: 0., fnc: 0.}

# get dynamics
x_next_m1 = x_next.subs(f_m1)
S1 = AffineSystem.from_symbolic(x, u, x_next_m1)

# build domain
D1 = Polyhedron.from_bounds(xu_min, xu_max)

# - gap <= 0 with floor and ceiling
gap_floor_m1 = gap_floor.subs(f_m1)
gap_ceiling_m1 = gap_ceiling.subs(f_m1)
D1.add_symbolic_inequality(xu, sp.Matrix([- gap_floor_m1]))
D1.add_symbolic_inequality(xu, sp.Matrix([- gap_ceiling_m1]))

# check domain
assert D1.bounded
assert not D1.empty

In [8]:
# discrete time dynamics in mode 2
# (ball sticking with the floor, not in contact with the ceiling)

# enforce sticking
fc_m2 = {ftc: 0., fnc: 0.}
ftf_m2 = sp.solve(sp.Eq(sliding_velocity_floor.subs(fc_m2), 0), ftf)[0]
fnf_m2 = sp.solve(sp.Eq(gap_floor.subs(fc_m2), 0), fnf)[0]
f_m2 = fc_m2.copy()
f_m2.update({ftf: ftf_m2, fnf: fnf_m2})

# get dynamics
x_next_m2 = x_next.subs(f_m2)
S2 = AffineSystem.from_symbolic(x, u, x_next_m2)

# build domain
D2 = Polyhedron.from_bounds(xu_min, xu_max)

# gap <= 0 with floor
D2.add_symbolic_inequality(xu, sp.Matrix([gap_floor_m1]))

# - gap <= 0 with ceiling
D2.add_symbolic_inequality(xu, sp.Matrix([- gap_ceiling_m1]))

# ball not falling down the floor
D2.add_symbolic_inequality(xu, ball_on_floor.subs(f_m2))

# friction cone
D2.add_symbolic_inequality(xu, sp.Matrix([ftf_m2 - mu*fnf_m2]))
D2.add_symbolic_inequality(xu, sp.Matrix([- ftf_m2 - mu*fnf_m2]))

# check domain
assert D2.bounded
assert not D2.empty

In [9]:
# discrete time dynamics in mode 3
# (ball sliding right on the floor, not in contact with the ceiling)

# enforce sticking
f_m3 = {ftf: -mu*fnf_m2, fnf: fnf_m2, ftc: 0., fnc: 0.}

# get dynamics
x_next_m3 = x_next.subs(f_m3)
S3 = AffineSystem.from_symbolic(x, u, x_next_m3)

# build domain
D3 = Polyhedron.from_bounds(xu_min, xu_max)

# gap <= 0 with floor
D3.add_symbolic_inequality(xu, sp.Matrix([gap_floor_m1]))

# - gap <= 0 with ceiling
D3.add_symbolic_inequality(xu, sp.Matrix([- gap_ceiling_m1]))

# ball not falling down the floor
D3.add_symbolic_inequality(xu, ball_on_floor.subs(f_m3))

# positive relative velocity
D3.add_symbolic_inequality(xu, sp.Matrix([- sliding_velocity_floor.subs(f_m3)]))

# check domain
assert D3.bounded
assert not D3.empty

In [10]:
# discrete time dynamics in mode 4
# (ball sliding left on the floor, not in contact with the ceiling)

# enforce sticking
f_m4 = {ftf: mu*fnf_m2, fnf: fnf_m2, ftc: 0., fnc: 0.}

# get dynamics
x_next_m4 = x_next.subs(f_m4)
S4 = AffineSystem.from_symbolic(x, u, x_next_m4)

# build domain
D4 = Polyhedron.from_bounds(xu_min, xu_max)

# gap <= 0 with floor
D4.add_symbolic_inequality(xu, sp.Matrix([gap_floor_m1]))

# - gap <= 0 with ceiling
D4.add_symbolic_inequality(xu, sp.Matrix([- gap_ceiling_m1]))

# ball not falling down the floor
D4.add_symbolic_inequality(xu, ball_on_floor.subs(f_m4))

# negative relative velocity
D4.add_symbolic_inequality(xu, sp.Matrix([sliding_velocity_floor.subs(f_m4)]))

# check domain
assert D4.bounded
assert not D4.empty

In [11]:
# discrete time dynamics in mode 5
# (ball sticking on the ceiling, not in contact with the floor)

# enforce sticking
ff_m5 = {ftf: 0., fnf: 0.}
ftc_m5 = sp.solve(sp.Eq(sliding_velocity_ceiling.subs(ff_m5), 0), ftc)[0]
fnc_m5 = sp.solve(sp.Eq(gap_ceiling.subs(ff_m5), 0), fnc)[0]
f_m5 = ff_m5.copy()
f_m5.update({ftc: ftc_m5, fnc: fnc_m5})

# get dynamics
x_next_m5 = x_next.subs(f_m5)
S5 = AffineSystem.from_symbolic(x, u, x_next_m5)

# build domain
D5 = Polyhedron.from_bounds(xu_min, xu_max)

# - gap <= 0 with floor
D5.add_symbolic_inequality(xu, sp.Matrix([- gap_floor_m1]))

# gap <= 0 with ceiling
D5.add_symbolic_inequality(xu, sp.Matrix([gap_ceiling_m1]))

# ball in contact with the ceiling
D5.add_symbolic_inequality(xu, ball_on_ceiling.subs(f_m5))

# friction cone
D5.add_symbolic_inequality(xu, sp.Matrix([ftc_m5 - mu*fnc_m5]))
D5.add_symbolic_inequality(xu, sp.Matrix([- ftc_m5 - mu*fnc_m5]))

# check domain
assert D5.bounded
assert not D5.empty

In [12]:
# discrete time dynamics in mode 6
# (ball sliding right on the ceiling, not in contact with the floor)


# enforce sticking
f_m6 = {ftc: -mu*fnc_m5, fnc: fnc_m5, ftf: 0., fnf: 0.}

# get dynamics
x_next_m6 = x_next.subs(f_m6)
S6 = AffineSystem.from_symbolic(x, u, x_next_m6)

# build domain
D6 = Polyhedron.from_bounds(xu_min, xu_max)

# - gap <= 0 with floor
D6.add_symbolic_inequality(xu, sp.Matrix([- gap_floor_m1]))

# gap <= 0 with ceiling
D6.add_symbolic_inequality(xu, sp.Matrix([gap_ceiling_m1]))

# ball in contact with the ceiling
D6.add_symbolic_inequality(xu, ball_on_ceiling.subs(f_m6))

# positive relative velocity
D6.add_symbolic_inequality(xu, sp.Matrix([- sliding_velocity_ceiling.subs(f_m6)]))

# check domain
assert D6.bounded
assert not D6.empty

In [13]:
# discrete time dynamics in mode 7
# (ball sliding left on the ceiling, not in contact with the floor)

# enforce sticking
f_m7 = {ftc: mu*fnc_m5, fnc: fnc_m5, ftf: 0., fnf: 0.}

# get dynamics
x_next_m7 = x_next.subs(f_m7)
S7 = AffineSystem.from_symbolic(x, u, x_next_m7)

# build domain
D7 = Polyhedron.from_bounds(xu_min, xu_max)

# - gap <= 0 with floor
D7.add_symbolic_inequality(xu, sp.Matrix([- gap_floor_m1]))

# gap <= 0 with ceiling
D7.add_symbolic_inequality(xu, sp.Matrix([gap_ceiling_m1]))

# ball in contact with the ceiling
D7.add_symbolic_inequality(xu, ball_on_ceiling.subs(f_m7))

# negative relative velocity
D7.add_symbolic_inequality(xu, sp.Matrix([sliding_velocity_ceiling.subs(f_m7)]))

# check domain
assert D7.bounded
assert not D7.empty

In [14]:
# list of dynamics
S_list = [S1, S2, S3, S4, S5, S6, S7]

# list of domains
D_list = [D1, D2, D3, D4, D5, D6, D7]

# PWA system
S = PieceWiseAffineSystem(S_list, D_list)

In [15]:
# controller parameters
N = 20
Q = np.diag([
    1., 1., .01,
    1., 1.,
    1., 1., .01,
    1., 1.
])*h
R = np.diag([
    .01, .001
])*h
P = np.zeros((S.nx, S.nx))

# terminal set and cost
X_N = Polyhedron.from_bounds(*[np.zeros(S.nx)]*2)

In [16]:
methods = [
    'Convex hull, lifted constraints',
    'Convex hull',
    'Big-M',
    'Traditional formulation'
]    
norms = ['inf', 'one', 'two']

In [17]:
# initial condition
x0 = np.array([
    0., 0., np.pi,
    0., 0.,
    0., 0., 0.,
    0., 0.
])

In [ ]:
sol = {}
for norm in norms:
    print 'norm:', norm
    controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, 'Convex hull', norm)
    sol[norm]= {}
    sol[norm]['u'], sol[norm]['x'], sol[norm]['ms'], sol[norm]['cost'] = controller.feedforward(x0)
# np.save('solution', sol)

In [ ]:
'''
norm inf:
objective 0.7076328888497
mode sequence [1, 1, 0, 5, 0, 0, 0, 3, 1, 1, 1, 1, 1, 0, 4, 0, 0, 0, 1, 1]

norm one:
objective 1.459892535120
mode sequence [1, 1, 0, 5, 0, 0, 0, 3, 1, 1, 1, 1, 1, 0, 4, 0, 0, 0, 1, 1]

norm two:
objective 0.9105320893042
mode sequence [1, 1, 0, 5, 0, 0, 3, 1, 1, 1, 1, 1, 1, 0, 4, 0, 0, 0, 1, 1]
'''

# Animation

In [ ]:
import meshcat
from meshcat.geometry import Box, Sphere, Cylinder, MeshLambertMaterial
from meshcat.animation import Animation
import meshcat.transformations as tf

In [ ]:
vis = meshcat.Visualizer()
# vis.jupyter_cell()
vis.open()

In [ ]:
tickness = .01
depth = .3
red = 0xff2222
blue = 0x2222ff
green = 0x22ff22
grey = 0x999999
vis['ball'].set_object(
    Sphere(r),
    MeshLambertMaterial(color=blue)
)
vis['floor'].set_object(
    Box([depth, l*2., tickness]),
    MeshLambertMaterial(color=red)
)
vis['ceiling'].set_object(
    Box([depth, l*2., tickness]),
    MeshLambertMaterial(color=grey)
)
vis['ball_orientation'].set_object(
    Cylinder(r/10., r*1.002),
    MeshLambertMaterial(color=green)
)

In [ ]:
anim = Animation()
for t, xt in enumerate(sol['two']['x']):
    with anim.at_frame(vis, t*h*30) as frame:
        frame['ball'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+r])
        )
        frame['floor'].set_transform(
            tf.translation_matrix([0, xt[3], xt[4]-tickness/2.])
        )
        frame['ceiling'].set_transform(
            tf.translation_matrix([0, 0, d+tickness/2.])
        )
        frame['ball_orientation'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+r]).dot(
                tf.rotation_matrix(xt[2], [1.,0.,0.])
            )
        )
vis.set_animation(anim)

# Compare formulations for different levels of relaxation

In [ ]:
# cost of each relaxation as a function of time
sol = np.load('solution_20_steps_new_cost.npy').item()
costs = {}
for norm in norms:
    print 'norm', norm
    costs[norm] = {}
    for method in methods:
        print 'method', method
        controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, method, norm)
        costs[norm][method] = []
        for ms in [sol[norm]['ms'][:i] for i in range(N+1)]:
            cost = controller.solve_relaxation(x0, ms)[1]
            if cost is not None:
                cost /= sol[norm]['cost']
            costs[norm][method].append(cost)

In [ ]:
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

In [ ]:
for norm in norms:
    mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')
#     plt.rc('font', size=14)
    colors = ['b', 'r', 'c','g']
    linestyles = ['-', '-.', '--', ':']
    for i, method in enumerate(methods):
        plt.plot(
            range(N+1),
            costs[norm][method],
            label=method,
            color=colors[i],
            linestyle=linestyles[i],
            linewidth=3
        )
    plt.xlim((0, N))
    plt.ylim((0, 1.1))
    plt.legend()
    plt.grid(True)
    if norm == 'inf':
        plt.title(r'Linear objective, $\infty$-norm')
    elif norm == 'one':
        plt.title(r'Linear objective, 1-norm')
    elif norm == 'two':
        plt.title(r'Quadratic objective')
    plt.xlabel(r'$t$')
    plt.ylabel(r'Cost relaxed problem / cost MICP')
    plt.savefig('relaxation_ratio_' + norm + '.pdf', bbox_inches='tight')
    plt.show()

# Objective as a function of the initial state ($x_1$ and $x_3$ only)

In [18]:
# samples for the initial position and velocity of the ball
n_levels = 10
n_samples = 101
residual_states = [0,2]
dropped_states = [i for i in range(S.nx) if i not in residual_states]
xb_samples = np.linspace(x_min[0], x_max[0], n_samples)
tb_samples = np.linspace(x_min[2], x_max[2], n_samples)

In [ ]:
# solve relaxations for all MI formulations
cost_on_feasible_set = {}

for method in methods:
    cost_on_feasible_set[method] = {}
    print 'method:', method
    if method == 'Convex hull, lifted constraints':
        controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, 'Convex hull', 'two')
    else:
        controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, method, 'two')

    # get feasible sets
    cs = get_constraint_set(controller.prog)
    var_indices = {v: i for i, v in enumerate(controller.prog.getVars())}
    dropped_indices = [var_indices[controller.prog.getVarByName('x0['+str(i)+']')] for i in dropped_states]
    residual_indices = [i for i in range(cs.A.shape[1]) if i not in dropped_indices]
    cs_sect = Polyhedron(cs.A[:, residual_indices], cs.b)
    proj = convex_hull_method_fast(cs_sect, [0,1])
    
    for norm in norms:
        print 'norm:', norm

        # samples on the grid
        controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, method, norm)
        cost_mat = np.empty([n_samples]*2)
        for i, xb in enumerate(xb_samples):
            for j, tb in enumerate(tb_samples):
                print(str(i) + ',' + str(j) + '   \r'),
                x0 = np.array([xb,0.,tb] + [0.]*7)
                cost_mat[i,j] = controller.solve_relaxation(x0, {})[1]

        # store data
        cost_on_feasible_set[method][norm] = {}
        cost_on_feasible_set[method][norm]['feasible_set'] = proj
        cost_on_feasible_set[method][norm]['cost_matrix'] = cost_mat
        
# save data
np.save('cost_on_feasible_set', cost_on_feasible_set)

method: Convex hull, lifted constraints
Academic license - for non-commercial use only
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
norm: inf
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [ ]:
def my_round(x):
    return floor(x*10.)/10.
methods = [
    'Big-M',
    'Traditional formulation'
]    

In [ ]:
# plot cost and feasible set
# plt.rc('font', size=14)
Xb, Tb = np.meshgrid(xb_samples, tb_samples)
cfs = np.load('cost_on_feasible_set.npy').item()
for method in methods:
    print 'method', method
    for norm in norms:
        print 'norm', norm
        cm = cfs[method][norm]['cost_matrix']
        fs = cfs[method][norm]['feasible_set']
        levels = [my_round((i+1)*np.nanmax(cm)/n_levels) for i in range(n_levels)]
        levels = [(i+1)*np.nanmax(cm)/n_levels for i in range(n_levels)]
        cp = plt.contour(Xb, Tb, cm.T, levels=levels, cmap='viridis_r')
        plt.colorbar(cp, label='Cost relaxed problem')
        fs.plot(facecolor='w')
        plt.xlabel(r'$x_1$')
        plt.ylabel(r'$x_3$')
        if norm == 'inf':
            plt.title(method + '\n' + r'Linear objective, $\infty$-norm')
        elif norm == 'one':
            plt.title(method + '\n' + r'Linear objective, 1-norm')
        elif norm == 'two':
            plt.title(method + '\n' + r'Quadratic objective')
        plt.grid(True)
    #     plt.savefig(method + '.pdf',bbox_inches='tight')
#         plt.savefig('new_ch.pdf',bbox_inches='tight')
        plt.show()

# Plot frames

In [ ]:
tickness = 0.02
def plot_ball(x, **kwargs):
    ax = plt.gca()
    ball = plt.Circle((x[0], x[1]+r), r, **kwargs)
    orientation = plt.plot(
        (x[0]-r*np.sin(x[2]), x[0]+r*np.sin(x[2])),
        (x[1]+r*(1.+np.cos(x[2])), x[1]+r*(1.-np.cos(x[2]))),
        color='k'
    )
    print (x[0]-r*np.sin(x[2]), x[1]+r*np.cos(x[2])),(x[0]+r*np.sin(x[2]), x[1]-r*np.cos(x[2]))
    ax.add_artist(ball)
def plot_floor(x, **kwargs):
    ax = plt.gca()
    floor = plt.Rectangle((x[3]-l/2., x[4]-tickness), l, tickness, **kwargs)
    ax.add_artist(floor)
def plot_ceiling(**kwargs):
    ax = plt.gca()
    ceiling = plt.Rectangle((-l/2., d), l, tickness, **kwargs)
    ax.add_artist(ceiling)
def plot_frame(x):
    plot_ball(x, facecolor='r', edgecolor='k')
    plot_floor(x, facecolor='g', edgecolor='k')
    plot_ceiling(facecolor='b', edgecolor='k')
    plt.axis('equal')
    plt.axis('off')
    plt.xlim([-.4,.4])
    plt.ylim([-.2,.6])
    plt.show()
for xt in x:
    plot_frame(xt)